In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from datasets import TransformedDataset
import utils
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
from models import MultiClassJetNet

torch.manual_seed(2)


In [2]:
# Default scalings for the default boxes determined by k-means clustering
default_scalings = torch.tensor(
    [
        [0.06549374, 0.12928654],
        [0.11965626, 0.26605093],
        [0.20708716, 0.38876095],
        [0.31018215, 0.47485098],
        [0.415882, 0.8048184],
        [0.7293086, 0.8216225],
    ]
)
encoder = utils.Encoder(default_scalings, ["robot"])
transformed_train_data = TransformedDataset("data/train", encoder)
transformed_val_data = TransformedDataset("data/val", encoder)


In [3]:
train_loader = DataLoader(
    transformed_train_data, batch_size=32, shuffle=True, num_workers=0
)
val_loader = DataLoader(
    transformed_val_data, batch_size=32, shuffle=False, num_workers=0
)

pl_model = MultiClassJetNet(encoder, 1e-3)
trainer = pl.Trainer(
    limit_predict_batches=100, max_epochs=200, callbacks=[RichProgressBar()]
)
trainer.fit(model=pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy            │ MulticlassAccuracy │      0 │
│ 1 │ input_layer         │ NormConv2dReLU     │    146 │
│ 2 │ depth_wise_backbone │ Sequential         │  4.9 K │
│ 3 │ classifier          │ Sequential         │ 20.9 K │
│ 4 │ output_layer        │ Conv2d             │  1.5 K │
└───┴─────────────────────┴────────────────────┴────────┘

Trainable params: 27.4 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 27.4 K                                                                                               
Total estimated model params size (MB): 0

2023-03-10 12:45:36.371893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 12:45:36.798255: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-10 12:45:38.624348: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-10 12:45:38.624552: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Output()

/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:22
4: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in
the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:22
4: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in
the `DataLoader` init to improve performance.
  rank_zero_warn(

`Trainer.fit` stopped: `max_epochs=200` reached.
